In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import pandas as pd

In [70]:
char_core = ['acc','agr','beta','bm','cash','cashpr','cfp','chatoia','chcsho','chfeps','chinv','chmom',
             'chpmia','chtx','currat','depr','dy','ear','ep','gma','grcapx','grltnoa','ill','indmom','invest',
             'lev','lgr','maxret','mom12m','mom1m','mom36m','mve','nincr','orgcap','pchgm_pchsale','pchsale_pchinvt',
             'pchsale_pchrect','pchsale_pchxsga','retvol','roaq','roavol','roeq','salecash','saleinv','sgr','sp',
             'std_dolvol','std_turn','turn']

core = ['date','permno','ticker','comnam','exchcd','exchname','siccd',
        'indname','size_class','mve_m','ret_adj_ex',]

#core = ['fyear','year','jyear','permno','ticker','comnam','exchcd','exchname','siccd',
#        'indname','size_class','mve_m','rf','ret','ret_adj','ret_ex','ret_adj_ex',]

print(len(char_core), len(core))

data = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Dataset/research_result/norm_df49.pkl')
data['date'] = pd.to_datetime(data.index).to_period('M')
data.sort_values(by='date', ascending=True, inplace=True)

49 11


In [79]:
X = data[char_core+['date']]
X_info = data[core]
Y = data[['predicted_return','date']]
print(X.shape, Y.shape)

(210205, 50) (210205, 2)


In [98]:
from keras.models import load_model
import tensorflow as tf

def my_metric_fn(y_true, y_pred):
    num = tf.reduce_mean(tf.square(y_true - y_pred))
    den = tf.reduce_mean(tf.square(y_true))
    return 1 - num / den

model = load_model('/content/drive/MyDrive/Colab Notebooks/Dataset/research_result/model2.h5',
                   custom_objects = {"my_metric_fn": my_metric_fn})

## Neural Network Model (Pretrain before year 2000)

In [88]:
x_train = X[(X['date'] < pd.Period((str(1995)+"-1"),freq='M'))]
y_train = Y[(Y['date'] < pd.Period((str(1995)+"-1"),freq='M'))]

x_val = X[(X['date'] >= pd.Period((str(1995)+"-1"),freq='M')) & (X['date'] <= pd.Period((str(1999)+"-12"),freq='M'))]
y_val = Y[(Y['date'] >= pd.Period((str(1995)+"-1"),freq='M')) & (Y['date'] <= pd.Period((str(1999)+"-12"),freq='M'))]

In [90]:
x_train=x_train.drop("date", axis=1)
y_train= y_train.drop("date", axis=1)
x_val=x_val.drop("date", axis=1)
y_val=y_val.drop("date", axis=1)

In [99]:
x_test_info = X_info[(X_info['date'] >= pd.Period((str(2000)+"-1"),freq='M'))]
x_test = X[(X['date'] >= pd.Period((str(2000)+"-1"),freq='M'))]
y_test = Y[(Y['date'] >= pd.Period((str(2000)+"-1"),freq='M'))]

x_test = x_test.drop("date", axis=1)
y_test = y_test.drop("date", axis=1)
model.evaluate(x_test, y_test)
y_NN2 = model.predict(x_test)
print(y_NN2.reshape(-1).mean(), y_NN2.reshape(-1).std())

2813/2813 [==============================] - 3s 1ms/step
0.0078965025 0.0049119988


In [100]:
(x_test_info.index == y_test.index).sum()

90007

## Linear Regression Model (Pretrain before year 2000)

In [102]:
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error

def np_metric_fn(y_true, y_pred):
    num = np.mean(np.square(y_true - y_pred))
    den = np.mean(np.square(y_true - np.zeros_like(y_true)))
    return 1 - num / den

x_all = np.concatenate((x_train, x_val))
y_all = np.concatenate((y_train, y_val))

regr = linear_model.LinearRegression()
regr.fit(x_all, y_all)
y_pred_linear = regr.predict(x_test)
mse = mean_squared_error(y_test, y_pred_linear)
r2 = np_metric_fn(y_test, y_pred_linear)
print(mse, r2)

0.01566968182931832 predicted_return    0.005432
dtype: float64


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


# Peformance Metrics

In [103]:
pred_result = x_test_info.copy()
pred_result.index = x_test.index

pred_result['y_real'] = y_test.values
pred_result['NN_pred_pretrain'] = y_NN2.reshape(-1)
pred_result['lin_pred_pretrain'] = y_pred_linear

In [144]:
stock_list = pred_result['permno'].unique()
month_list = pred_result.index.unique()
print(len(stock_list), len(month_list))

607 269


In [173]:
pred_result.to_pickle("senario_1_result.pkl")

## XS - R2

In [164]:
# Metrics: XS-R^2 of Neural Network Model  (weighted)
r2_stock = {}
num_ = []
deno_ = []
for i in stock_list:
  df = pred_result[pred_result['permno'] == i]
  num_month = df.shape[0]
  num = np.square((df['y_real'] -  df['NN_pred_pretrain']).sum() / num_month) * num_month
  deno = np.square((df['NN_pred_pretrain']).sum() / num_month)  * num_month
  num_.append(num)
  deno_.append(deno)
print(1 - np.mean(num_) / np.mean(deno_))

-0.8150487244022571


In [165]:
# Metrics: XS-R^2 of Linear Regresion (weighted)
r2_stock = {}
num_ = []
deno_ = []
for i in stock_list:
  df = pred_result[pred_result['permno'] == i]
  num_month = df.shape[0]
  num = np.square((df['y_real'] -  df['lin_pred_pretrain']).sum() / num_month) * num_month
  deno = np.square((df['lin_pred_pretrain']).sum() / num_month)  * num_month
  num_.append(num)
  deno_.append(deno)
print(1 - np.mean(num_) / np.mean(deno_))

-1.6860012050668507


In [166]:
# Metrics: XS-R^2 of Neural Network Model  (NO weighted)
r2_stock = {}
num_ = []
deno_ = []
for i in stock_list:
  df = pred_result[pred_result['permno'] == i]
  num_month = df.shape[0]
  num = np.square((df['y_real'] -  df['NN_pred_pretrain']).sum() / num_month)
  deno = np.square((df['NN_pred_pretrain']).sum() / num_month)
  num_.append(num)
  deno_.append(deno)
print(1 - np.mean(num_) / np.mean(deno_))

-16.658326364290787


In [167]:
# Metrics: XS-R^2 of Linear Regresion (NO weighted)
r2_stock = {}
num_ = []
deno_ = []
for i in stock_list:
  df = pred_result[pred_result['permno'] == i]
  num_month = df.shape[0]
  num = np.square((df['y_real'] -  df['lin_pred_pretrain']).sum() / num_month)
  deno = np.square((df['lin_pred_pretrain']).sum() / num_month)
  num_.append(num)
  deno_.append(deno)
print(1 - np.mean(num_) / np.mean(deno_))

-30.426142440845414


## EV

In [163]:
# Metrics: EV of Neural Network
r2_stock = {}
num_ = []
deno_ = []
for i in month_list:
  df = pred_result[pred_result.index == i]
  num_stock = df.shape[0]
  num = (np.square(df['y_real'] -  df['NN_pred_pretrain'])).sum() / num_stock
  deno = (np.square(df['y_real'])).sum() /num_stock
  num_.append(num)
  deno_.append(deno)
  break
print(1 - np.mean(num_) / np.mean(deno_))

0.007265052040344333


In [162]:
# Metrics: EV of Linear Model
r2_stock = {}
num_ = []
deno_ = []
for i in month_list:
  df = pred_result[pred_result.index == i]
  num_stock = df.shape[0]
  num = (np.square(df['y_real'] -  df['lin_pred_pretrain'])).sum() / num_stock
  deno = (np.square(df['y_real'])).sum() /num_stock
  num_.append(num)
  deno_.append(deno)
  break
print(1 - np.mean(num_) / np.mean(deno_))

0.004059510189341364


## Total R2

In [168]:
# R^2 of Linear Regresion
a = np.sum(np.square(pred_result['y_real'] -  pred_result['lin_pred_pretrain']))
b = np.sum(np.square(pred_result['y_real']))
1-a/b

0.005432492063096261

In [169]:
# R^2 of Neural Network
a = np.sum(np.square(pred_result['y_real'] -  pred_result['NN_pred_pretrain']))
b = np.sum(np.square(pred_result['y_real']))
1-a/b

0.007359925494608732

## MSE

In [171]:
## MSE

mse_Linear = mean_squared_error(pred_result['y_real'], pred_result['lin_pred_pretrain'])
mse_NN = mean_squared_error(pred_result['y_real'], pred_result['NN_pred_pretrain'])
print(f"MSE of Linear model: {mse_Linear}")
print(f"MSE of Neural model: {mse_NN}")

MSE of Linear model: 0.01566968182931832
MSE of Neural model: 0.01563931459091774
